In [ ]:
# ==================================================================================
# Diplomado en Estadística Aplicada a la Toma de Decisiones con Lenguaje R y Python
# Universidad Privada Boliviana
# ----------------------------------------------------------------------------------
#   MODELOS PREDICTIVOS PARA LA TOMA DE DECISIONES ESTRATEGICAS
# ----------------------------------------------------------------------------------
#         Enrique Alejandro Laurel Cossio, Mayo 2025
# ==================================================================================
#              Métricas de Evaluación de Ajuste
# ==================================================================================

In [73]:
# Cargamos Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score,confusion_matrix,recall_score,classification_report,roc_curve

# 1. REGRESIÓN

In [54]:
# cargamos datos
url='https://raw.githubusercontent.com/ealaurel/MODELOS_PREDICTIVOS_202505/main/data/insurance.csv'
seguros = pd.read_csv(url,sep=',', encoding='iso-8859-1')

print(seguros.shape) #
seguros.head(2)

(1338, 7)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [55]:
# una copia de la fuente de datos
df = seguros.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [56]:
# a dummies las variables categoricas
df = pd.get_dummies(df, columns=['region'], drop_first=True)
print(df.shape)
df.head(2)

(1338, 9)


,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest
0,19,female,27.90,0,yes,16884.9240,False,False,True
1,18,male,33.77,1,no,1725.5523,False,True,False


In [57]:
# a dummies
df = pd.get_dummies(df, columns=['sex','smoker'], drop_first=True)
print(df.shape)
df.head(2)

(1338, 9)


,age,bmi,children,charges,region_northwest,region_southeast,region_southwest,sex_male,smoker_yes
0,19,27.90,0,16884.9240,False,False,True,False,True
1,18,33.77,1,1725.5523,False,True,False,True,False


In [58]:
# variables dummies  a entero
df = df.replace({True: 1, False: 0})

<ipython-input-58-fd397598af70>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [59]:
# Definir las variables independientes y dependientes
X = df[['age', 'sex_male', 'bmi', 'children', 'smoker_yes', 'region_northwest','region_southeast','region_southwest']]
y = df['charges'] # variables dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) # relacionado intercepto

In [60]:
# División de conjunto de datos en entrenamiento y testeo
# train - test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 22) #Separamos 30% para test

In [61]:
# Ajustar el modelo de regresión lineal
modelo_OLS = sm.OLS(y_train, x_train).fit()

# Resumen del modelo
print(modelo_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     354.2
Date:                Mon, 26 May 2025   Prob (F-statistic):          9.23e-276
Time:                        22:59:56   Log-Likelihood:                -9456.6
No. Observations:                 936   AIC:                         1.893e+04
Df Residuals:                     927   BIC:                         1.897e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.299e+04   1148.340  

In [62]:
# calculamos predicción
y_pred = modelo_OLS.predict(x_test)
y_pred


,0
1231,22825.346680
768,17845.585097
847,10573.771782
510,13421.544277
363,2158.505244
...,...
980,10437.256318
204,6088.693553
553,13170.748172
516,4211.782520


In [63]:
# R cuadrado del modelo
r2 = r2_score(y_test, y_pred)
print(f"R cuadrado (R^2): {r2:.4f}")

R cuadrado (R^2): 0.7398


- R² = 1: El modelo explica el 100% de la variabilidad de los datos. Es un **ajuste perfecto** (aunque esto puede indicar sobreajuste).
- R² > 0.8: Se considera un **ajuste muy bueno**. El modelo explica la mayor parte de la variabilidad.
- 0.5 ≤ R² ≤ 0.8: **Ajuste moderado**. El modelo explica parte de la variabilidad, pero hay factores no considerados.
- R² < 0.5: **Ajuste pobre**. El modelo no explica bien la variabilidad de los datos.
- R² = 0: El modelo no tiene capacidad predictiva.


In [64]:
# Error Cuadratico Medio
mse = mean_squared_error(y_test, y_pred)
print(f"Error cuadrático medio (MSE): {mse:.4f}")
# Mientras menos mejor

Error cuadrático medio (MSE): 40986218.2527


In [65]:
# Raiz del Error cuadratico Medio
rmse = np.sqrt(mse)
print(f"Raíz del error cuadrático medio (RMSE): {rmse:.4f}")

Raíz del error cuadrático medio (RMSE): 6402.0480


In [71]:
# Coeficiente de Variación del RMSE
cv_rmse = (rmse / np.mean(y_test)) * 100
print(f"Coeficiente de variación del RMSE: {cv_rmse:.2f}%")


Coeficiente de variación del RMSE: 46.40%


# 2. CLASIFICACIÓN

In [76]:
# cargamos datos
url='https://raw.githubusercontent.com/ealaurel/MODELOS_PREDICTIVOS_202505/main/data/ifood_df.csv'
ifood_df = pd.read_csv(url,sep=',', encoding='iso-8859-1')

print(ifood_df.shape) #
ifood_df.head(2)

(2205, 39)


,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,marital_Together,marital_Widow,education_2n Cycle,education_Basic,education_Graduation,education_Master,education_PhD,MntTotal,MntRegularProds,AcceptedCmpOverall
0,58138.0,0,0,58,635,88,546,172,88,88,...,0,0,0,0,1,0,0,1529,1441,0
1,46344.0,1,1,38,11,1,6,2,1,6,...,0,0,0,0,1,0,0,21,15,0


In [77]:
# una copia a los datos
df = ifood_df.copy()

In [78]:
df.columns

Index(['Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response',
       'Age', 'Customer_Days', 'marital_Divorced', 'marital_Married',
       'marital_Single', 'marital_Together', 'marital_Widow',
       'education_2n Cycle', 'education_Basic', 'education_Graduation',
       'education_Master', 'education_PhD', 'MntTotal', 'MntRegularProds',
       'AcceptedCmpOverall'],
      dtype='object')

In [79]:
# seleccionamos variables explicativas y variables dependiente
X = df[['MntRegularProds','Kidhome','Recency','education_Basic']]
y = df['Response']

In [80]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

In [81]:
# reiniciamos indice por cada data frame
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [82]:
# Ajusta el modelo de regresión logística
modelo_logit = sm.Logit(y_train, X_train).fit()

Optimization terminated successfully.
         Current function value: 0.389095
         Iterations 7


In [83]:
# Muestra un resumen del modelo
print(modelo_logit.summary())

                           Logit Regression Results                           
Dep. Variable:               Response   No. Observations:                 1764
Model:                          Logit   Df Residuals:                     1760
Method:                           MLE   Df Model:                            3
Date:                Mon, 26 May 2025   Pseudo R-squ.:                 0.05592
Time:                        23:14:35   Log-Likelihood:                -686.36
converged:                       True   LL-Null:                       -727.02
Covariance Type:            nonrobust   LLR p-value:                 1.606e-17
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
MntRegularProds     0.0006      0.000      5.248      0.000       0.000       0.001
Kidhome            -0.5346      0.129     -4.145      0.000      -0.787      -0.282
Recency            -0.0399      

In [84]:
# prediccion
y_pred_train = modelo_logit.predict(X_train)
y_pred_train.head(2)

,0
0,0.044692
1,0.132697


In [106]:
import numpy as np
from sklearn.metrics import precision_recall_curve, f1_score
# corte óptimo F1 Score para la variable respuesta
def mejor_corte_f1(y_true, y_scores):
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_scores)
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
    mejor_umbral = thresholds[np.argmax(f1_scores[:-1])]  # Se excluye el último valor de F1 Score

    return mejor_umbral

In [107]:
# Ejemplo de uso
y_train # Etiquetas reales
y_pred_train # Probabilidades del modelo

mejor_umbral = mejor_corte_f1(y_train, y_pred_train)
print(f"El mejor punto de corte según el F1 score es: {mejor_umbral:.2f}")

El mejor punto de corte según el F1 score es: 0.18


In [108]:
# Calcula la Exactitud del modelo en el entrenamiento (umbral cualesquiera)
# y_train: valores verdadaderos
# y_pred: valores predichos
exactitud_train = accuracy_score(y_train, (y_pred_train>0.5).astype(int))
print("Exactitud o accuracy del modelo:", exactitud_train)

Exactitud o accuracy del modelo: 0.8395691609977324


In [109]:
# Calcula la Exactitud del modelo en el entrenamiento (umbral optimo F1 Score)
# y_train: valores verdadaderos
# y_pred: valores predichos
exactitud_train = accuracy_score(y_train, (y_pred_train>mejor_umbral).astype(int))
print("Exactitud o accuracy del modelo:", exactitud_train)

Exactitud o accuracy del modelo: 0.655328798185941


In [110]:
# Realiza predicciones en el conjunto de datos de prueba
y_pred = modelo_logit.predict(X_test)
y_pred

,0
0,0.065869
1,0.017433
2,0.176872
3,0.175467
4,0.071457
...,...
436,0.066806
437,0.403941
438,0.641312
439,0.090864


In [105]:
# Matriz de confusión (umbral 0.5)
confusion = confusion_matrix(y_test, (y_pred>0.50).astype(int))
print("Matriz de confusión:")
print(confusion)
            #Predicción
            #   0   1
#ValorReal  0 # VN FP
#Valor Real 1 # FN VP

Matriz de confusión:
[[347  15]
 [ 65  14]]


In [111]:
# Matriz de confusión (umbral F1 score)
confusion = confusion_matrix(y_test, (y_pred>mejor_umbral).astype(int))
print("Matriz de confusión:")
print(confusion)
            #Predicción
            #   0   1
#ValorReal  0 # VN FP
#Valor Real 1 # FN VP

Matriz de confusión:
[[230 132]
 [ 29  50]]


In [112]:
## Exactitud o Accuracy del modelo
#exactitud_test = accuracy_score(y_test, (y_pred>0.5).astype(int))
#print("Precisión del modelo:", exactitud_test)

In [ ]:
# PRECISIÓN: Porcentaje de verdaros positivos en la clase de predicción postiva VP/(VP+FP)
# RECALL: Porcentaje de verdaderos postivos sobre el total de la clase positiva real VP/(VP +FN)

In [116]:
# Medidas Evaluacion (corte 0.5)
med_eval_05 = classification_report(y_test, (y_pred>0.5).astype(int))
print(med_eval_05)

              precision    recall  f1-score   support

           0       0.84      0.96      0.90       362
           1       0.48      0.18      0.26        79

    accuracy                           0.82       441
   macro avg       0.66      0.57      0.58       441
weighted avg       0.78      0.82      0.78       441



In [117]:
# Medidas Evaluacion (corte F1)
med_eval_F1 = classification_report(y_test, (y_pred>mejor_umbral).astype(int))
print(med_eval_F1)

              precision    recall  f1-score   support

           0       0.89      0.64      0.74       362
           1       0.27      0.63      0.38        79

    accuracy                           0.63       441
   macro avg       0.58      0.63      0.56       441
weighted avg       0.78      0.63      0.68       441

